In [ ]:
!pip uninstall tensorflow 
!pip install tensorflow-gpu==1.14

In [ ]:
!pip uninstall keras 
!pip install keras==2.2.4

In [ ]:
pip install h5py==2.10.0 --force-reinstall #need to install h5 ,otherwise it won't load the model properly

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
!cp /content/gdrive/MyDrive/Flood_Project/Image_Stylization/instancenormalization.py /content

In [ ]:
from instancenormalization import InstanceNormalization 

Using TensorFlow backend.


In [ ]:
import os
import datetime
import keras
from keras.preprocessing.image import *
from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras_preprocessing.image import *
from keras import backend as K
from keras.callbacks import TensorBoard
from keras import losses
import random
import h5py

In [ ]:

class CycleGAN():

    def __init__(self):

        self.sess = K.get_session()
        self.log_path = '/content/gdrive/MyDrive/Flood_Project/Image_Stylization/logs_2'
        self.domain_A_data_dir = '/content/gdrive/MyDrive/Flood_Project/Image_Stylization/Synthetic_to_Real/data/'
        self.domain_B_data_dir = '/content/gdrive/MyDrive/Flood_Project/Image_Stylization/Synthetic_to_Real/data/'
        self.f_size = 3
        self.first_f_size = 7
        self.d_f_size = 3
        self.leakyrel_alpha = 0.2
        self.img_height = 256
        self.img_width = 256
        self.img_channels = 3
        self.discr_dropout_rate = 0.0
        self.resblock_dropout = 0.5
        self.img_shape = (self.img_height, self.img_width, self.img_channels)
        self.image_data_generator = ImageDataGenerator(horizontal_flip=True)
        self.save_gens = 100
        self.batch_size = 1

        self.domain_A_datagen = self.image_data_generator.flow_from_directory(directory=self.domain_A_data_dir,
                                                                              target_size=(
                                                                              self.img_height, self.img_width),
                                                                              color_mode='rgb', classes=None,
                                                                              class_mode=None,
                                                                              batch_size=self.batch_size)
        self.domain_B_datagen = self.image_data_generator.flow_from_directory(directory=self.domain_B_data_dir,
                                                                              target_size=(
                                                                              self.img_height, self.img_width),
                                                                              color_mode='rgb', classes=None,
                                                                              class_mode=None,
                                                                              batch_size=self.batch_size)

        patch = int(self.img_height/2**4)
        self.disc_patch = (patch, patch, 1)

        self.g_f_ch = 32
        self.d_f_ch = 64

        self.lambda_perceptual_cycle_loss_wt = 10.0
        self.lambda_id_wt = 0.1 * self.lambda_perceptual_cycle_loss_wt
        self.mse_loss_wt = 1.0

        self.sample_modulus = 100
        self.id_modulus = 10
        self.stop_id = 100
        self.flip_labels_modulus = 0

        d_optimizer = Adam(lr=0.0002, beta_1=0.5)
        optimizer = Adam(lr=0.0002, beta_1=0.5)

        self.perceptual_model = self.build_perceptual_model()

        self.Disc_A = self.build_discriminator()
        self.Disc_B = self.build_discriminator()

        self.Disc_A.compile(loss=['mse'],
                            optimizer=d_optimizer, metrics=['accuracy'])
        self.Disc_B.compile(loss=['mse'],
                            optimizer=d_optimizer, metrics=['accuracy'])

        self.Gen_AB = self.build_generator()
        self.Gen_BA = self.build_generator()

        img_A = Input(shape=self.img_shape, name='image_A')
        img_B = Input(shape=self.img_shape, name='image_B')


        fake_B = self.Gen_AB([img_A])
        fake_A = self.Gen_BA([img_B])

        reconstruct_A = self.Gen_BA([fake_B])
        reconstruct_B = self.Gen_AB([fake_A])

        rec_content_A = self.perceptual_model(reconstruct_A)
        rec_content_B = self.perceptual_model(reconstruct_B)

        # Identity mapping of img

        img_A_id = self.Gen_BA([img_A])
        img_B_id = self.Gen_AB([img_B])

        self.Disc_A.trainable = False
        self.Disc_B.trainable = False

        valid_A = self.Disc_A(fake_A)
        valid_B = self.Disc_B(fake_B)

        comb_outputs = [valid_A, valid_B,
                    rec_content_A, rec_content_B
                    ]

        self.combined = Model(inputs=[img_A, img_B],
                        outputs=comb_outputs)

        self.combined.compile(
            loss=['mse', 'mse',
                  'mae', 'mae'
                  ],
            optimizer=optimizer,
            loss_weights = [self.mse_loss_wt, self.mse_loss_wt,
                            self.lambda_perceptual_cycle_loss_wt,
                            self.lambda_perceptual_cycle_loss_wt,
                            ])

        self.id_trainer = Model(inputs=[img_A, img_B],
                                outputs=[img_A_id, img_B_id])

        self.id_trainer.compile(loss=['mae', 'mae'],
                                optimizer=optimizer, loss_weights=[self.lambda_id_wt, self.lambda_id_wt])

    def build_perceptual_model(self):
        vgg = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(self.img_shape))
        content_layer = ['block5_conv3']
        #style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
        # Get output layers corresponding to style and content layers
        content_outputs = [vgg.get_layer(name).output for name in content_layer ]
        model = Model(vgg.input, content_outputs)
        model.trainable = False
        model.summary()
        return model

    def build_generator(self):
        """U-Net/ Resblock Generator"""

        def conv2d(layer_input, filters, f_size=self.f_size, s=2):
            """Layers used during downsampling"""
            d_1 = Conv2D(filters, kernel_size=f_size, strides=s, padding='same')(layer_input)
            d_1 = LeakyReLU(self.leakyrel_alpha)(d_1)
            d = InstanceNormalization()(d_1)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=self.f_size, dropout_rate=0.0):
            """Layers used during upsampling"""
            u = UpSampling2D()(layer_input)
            u = Conv2D(filters=filters, kernel_size=f_size, strides=1, activation='relu', padding='same')(u)
            u = InstanceNormalization()(u)
            if skip_input is not None:
                u = Concatenate()([u, skip_input])
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            return u

        def residual_block(x, filters, f_size, dropout=self.resblock_dropout):
            if K.shape(x)[2] == filters:
                x = x
            else:
                x = Conv2D(filters=filters, kernel_size=f_size, strides=(1, 1), padding='same', activation='relu')(x)
                x = InstanceNormalization()(x)
            conv1 = Conv2D(filters=filters, kernel_size=f_size, strides=(1, 1), padding='same', activation='relu')(x)
            inst_norm = InstanceNormalization()(conv1)
            if dropout:
                inst_norm = GaussianDropout(dropout)(inst_norm)
            conv2 = Conv2D(filters=filters, kernel_size=f_size, strides=(1, 1), padding='same')(inst_norm)
            x = Add()([x, conv2])
            x = Activation('relu')(x)
            x = InstanceNormalization()(x)
            return x

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.g_f_ch, f_size=self.first_f_size, s=1)
        d1 = GaussianNoise(0.01)(d1)

        d2 = conv2d(d1, self.g_f_ch * 2)
        d3 = conv2d(d2, self.g_f_ch * 4)
        d4 = conv2d(d3, self.g_f_ch * 8)


        t1 = residual_block(d4, filters=self.g_f_ch * 8, f_size=self.f_size)
        t2 = residual_block(t1, filters=self.g_f_ch * 8, f_size=self.f_size)
        t3 = residual_block(t2, filters=self.g_f_ch * 8, f_size=self.f_size)
        t4 = residual_block(t3, filters=self.g_f_ch * 8, f_size=self.f_size)
        t5 = residual_block(t4, filters=self.g_f_ch * 8, f_size=self.f_size)
        t5 = GaussianNoise(0.01)(t5)
        t6 = residual_block(t5, filters=self.g_f_ch * 8, f_size=self.f_size)
        t7 = residual_block(t6, filters=self.g_f_ch * 8, f_size=self.f_size)
        t8 = residual_block(t7, filters=self.g_f_ch * 8, f_size=self.f_size)
        t9 = residual_block(t8, filters=self.g_f_ch * 8, f_size=self.f_size)
        t10 = residual_block(t9, filters=self.g_f_ch * 8, f_size=self.f_size)

        # Upsampling
        u1 = deconv2d(t10, skip_input=d3, filters=self.g_f_ch * 4)
        u2 = deconv2d(u1, skip_input=d2, filters=self.g_f_ch * 2)
        u3 = UpSampling2D(2)(u2)
        output_img = Conv2D(self.img_channels, kernel_size=self.first_f_size, strides=1, padding='same', activation='tanh')(u3)



        return Model([d0], output_img)

    # Discriminator

    def build_discriminator(self):

        def d_layer(layer_input, filters, dropout=self.discr_dropout_rate, f_size=self.d_f_size, normalization=True, s=2, dilation_rate=1):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=s, padding='same', dilation_rate=dilation_rate)(layer_input)
            d = LeakyReLU(self.leakyrel_alpha)(d)
            if normalization:
                d = InstanceNormalization()(d)
            if dropout:
                d = Dropout(dropout)(d)
            return d

        img = Input(shape=self.img_shape)
        d1 = d_layer(img, self.d_f_ch, f_size=self.first_f_size, normalization=False, s=1)
        d2 = d_layer(d1, self.d_f_ch * 2)
        d3 = d_layer(d2, self.d_f_ch * 4)
        d4 = d_layer(d3, self.d_f_ch * 8)
        d5 = d_layer(d4, self.d_f_ch * 16, dropout=0.05)
        validity = Conv2D(1, kernel_size=self.d_f_size, strides=1, padding='same', activation='linear')(d5)
        return Model(img, [validity])

    def write_log(self, callback, names, logs, batch_no):
        for name, value in zip(names, logs):
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value
            summary_value.tag = name
            callback.writer.add_summary(summary, batch_no)
            callback.writer.flush()

    def train(self, epochs):


        self.combined.summary()

        start_time = datetime.datetime.now()
        batch_size = self.batch_size

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch) * 0.9
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            epoch = epoch+1

            imgs_A = self.domain_A_datagen.next()
            imgs_B = self.domain_B_datagen.next()

            imgs_A = (imgs_A*1/127.5)-1
            imgs_B = (imgs_B*1/127.5)-1

            # ----------------------
            #  Train Discriminators
            # ----------------------
            if self.flip_labels_modulus > 0 and epoch % self.flip_labels_modulus == 0:
                valid = np.zeros((batch_size,) + self.disc_patch)
                fake = np.ones((batch_size,) + self.disc_patch) * 0.9

            # Translate images to opposite domain
            fake_B_d_train = self.Gen_AB.predict([imgs_A])
            fake_A_d_train = self.Gen_BA.predict([imgs_B])

            # Train the discriminators (original images = real / translated = Fake)

            dA_loss_real = self.Disc_A.train_on_batch(imgs_A, [valid])
            dA_loss_fake = self.Disc_A.train_on_batch(fake_A_d_train, [fake])
            dA_loss = 0.5 * np.add(np.mean(dA_loss_real), np.mean(dA_loss_fake))

            dB_loss_real = self.Disc_B.train_on_batch(imgs_B, [valid])
            dB_loss_fake = self.Disc_B.train_on_batch(fake_B_d_train, [fake])
            dB_loss = 0.5 * np.add(np.mean(dB_loss_real), np.mean(dB_loss_fake))

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)

            # ------------------
            #  Train Generators
            # ------------------
            # Train the generators

            fake_B = self.Gen_AB.predict([imgs_A])
            fake_A = self.Gen_BA.predict([imgs_B])

            content_A = self.perceptual_model.predict_on_batch(imgs_A)
            content_B = self.perceptual_model.predict_on_batch(imgs_B)

            g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                    [valid, valid,
                                                    content_A, content_B
                                                    ])

            elapsed_time = datetime.datetime.now() - start_time

            adversarial_g_loss = np.mean((g_loss[0:1]))
            perceptual_reconstruction_loss = np.mean((g_loss[2:]))

            # Print the progress

            print("[Epoch %d/%d] [D loss: %05f] [Adversarial_G_Loss: %05f, perc_recon: %05f] time: %s"
                        %(epoch, epochs,
                        d_loss, adversarial_g_loss,
                        perceptual_reconstruction_loss, str(elapsed_time)))

            if self.id_modulus > 0 and epoch % self.id_modulus == 0 and epoch < self.stop_id:
                id_loss = self.id_trainer.train_on_batch(x=[imgs_A, imgs_B], y=[imgs_A, imgs_B])
                print("[ID_loss_A: %05f, ID_loss_B: %05f, ID_loss_A_B: %05f] " %(id_loss[0], id_loss[1], np.mean((id_loss))))

            if self.sample_modulus > 0 and epoch % self.sample_modulus == 0:
                imgs_frame = np.concatenate([imgs_A, fake_B, imgs_B, fake_A])
                imgs_frame = imgs_frame.reshape(batch_size * 4 * self.img_height, self.img_width, self.img_channels)
                imgs_frame = array_to_img(imgs_frame)
                imgs_frame.save('/content/gdrive/MyDrive/Flood_Project/Image_Stylization/log_5/epoch_%d.jpg' % (epoch))

            # Reset data_gens / shuffle
            if self.domain_A_datagen.n == self.domain_A_datagen.batch_index:
                self.domain_A_datagen.reset()
                self.domain_A_datagen.on_epoch_end()

            if self.domain_B_datagen.n == self.domain_B_datagen.batch_index:
                self.domain_B_datagen.reset()
                self.domain_B_datagen.on_epoch_end()

            if self.save_gens > 0 and epoch % self.save_gens == 0:
                self.Gen_AB.save('/content/gdrive/MyDrive/Flood_Project/Image_Stylization/log_5/gen_ab_v2_%d.h5' %(epoch))
                self.Gen_BA.save('/content/gdrive/MyDrive/Flood_Project/Image_Stylization/log_5/gen_ba_v2_%d.h5' %(epoch))


if __name__ == '__main__':
    gan = CycleGAN()
    gan.train(epochs=10000)

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
__________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 1/10000] [D loss: 25.943441] [Adversarial_G_Loss: 403.360413, perc_recon: 43.046631] time: 0:01:07.036712


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 2/10000] [D loss: 56.887312] [Adversarial_G_Loss: 89.013367, perc_recon: 22.991823] time: 0:01:08.198792
[Epoch 3/10000] [D loss: 70.153793] [Adversarial_G_Loss: 126.972748, perc_recon: 27.417517] time: 0:01:09.627503
[Epoch 4/10000] [D loss: 24.004235] [Adversarial_G_Loss: 24.703861, perc_recon: 3.113143] time: 0:01:10.796615
[Epoch 5/10000] [D loss: 2.712669] [Adversarial_G_Loss: 13.092405, perc_recon: 2.042856] time: 0:01:20.669258
[Epoch 6/10000] [D loss: 2.169471] [Adversarial_G_Loss: 6.846816, perc_recon: 0.485273] time: 0:01:21.499386
[Epoch 7/10000] [D loss: 0.646737] [Adversarial_G_Loss: 7.896449, perc_recon: 0.485524] time: 0:01:22.333555
[Epoch 8/10000] [D loss: 0.603688] [Adversarial_G_Loss: 6.452048, perc_recon: 0.583703] time: 0:01:23.138864
[Epoch 9/10000] [D loss: 1.020990] [Adversarial_G_Loss: 7.574659, perc_recon: 0.676945] time: 0:01:24.262293
[Epoch 10/10000] [D loss: 0.720233] [Adversarial_G_Loss: 8.355481, perc_recon: 0.529748] time: 0:01:25.177479
[ID_loss

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
[Epoch 5001/10000] [D loss: 0.287936] [Adversarial_G_Loss: 2.917990, perc_recon: 0.262089] time: 1:10:11.071741
[Epoch 5002/10000] [D loss: 0.303456] [Adversarial_G_Loss: 2.793170, perc_recon: 0.250140] time: 1:10:11.852811
[Epoch 5003/10000] [D loss: 0.287179] [Adversarial_G_Loss: 2.972234, perc_recon: 0.314506] time: 1:10:12.573565
[Epoch 5004/10000] [D loss: 0.309340] [Adversarial_G_Loss: 2.583922, perc_recon: 0.277160] time: 1:10:13.301715
[Epoch 5005/10000] [D loss: 0.307905] [Adversarial_G_Loss: 2.970917, perc_recon: 0.327959] time: 1:10:14.089706
[Epoch 5006/10000] [D loss: 0.301034] [Adversarial_G_Loss: 2.780952, perc_recon: 0.280377] time: 1:10:14.848544
[Epoch 5007/10000] [D loss: 0.290863] [Adversarial_G_Loss: 2.618255, perc_recon: 0.337968] time: 1:10:15.677643
[Epoch 5008/10000] [D loss: 0.302501] [Adversarial_G_Loss: 3.130245, perc_recon: 0.243981] time: 1:10:16.399541
[Epoch 5009/10000] [D loss: 0.276579] [Adversarial_G_